In [14]:
import os
import cv2
import glob
import pickle
import requests
import numpy as np
from io import BytesIO
from matplotlib import cm
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageDraw, ImageEnhance

from tensorflow.keras.preprocessing import image
from tensorflow.keras.backend import resize_images
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.resnet50 import ResNet50
# from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
# from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications.nasnet import preprocess_input, decode_predictions, NASNetLarge

In [ ]:
fname = "kay_images.npz"
with np.load(fname) as dobj:
    dat = dict(**dobj)

In [ ]:
dat.keys()

In [ ]:
im = Image.fromarray(np.uint8(cm.gist_earth(dat["stimuli"][0])*255)).convert("L")
plt.imshow(dat["stimuli"][0], cmap="gray")

In [ ]:
f, axs = plt.subplots(2, 4, figsize=(12, 6), sharex=True, sharey=True)
for ax, im in zip(axs.flat, dat["stimuli"]):
    ax.imshow(im, cmap="gray")
f.tight_layout()

In [ ]:
dat["stimuli"][0].shape

### stimuli images pre-process (crop circle, resize and colorize)

In [ ]:
def crop_resize_image_circle(im):
    size = (256, 256) # (128, 128)
    mask = Image.new('L', size, 0)
    draw = ImageDraw.Draw(mask) 
    draw.ellipse((0, 0) + size, fill=255)
    im = Image.fromarray(np.uint8(cm.gist_earth(im)*255)).convert("L")
    im.thumbnail(size, Image.ANTIALIAS)

    output = ImageOps.fit(im, mask.size, centering=(0.5, 0.5))
    output.putalpha(mask)
#     enhancer = ImageEnhance.Brightness(output)
#     output = enhancer.enhance(1.8)
#     output.save('output.png')
    return output

In [ ]:
# for i, im in tqdm_notebook(enumerate(dat["stimuli"])):
for i, im in tqdm_notebook(enumerate(dat["stimuli_test"])):
#     if i > 20: break
#     cim = crop_resize_image_circle(im)
#     print(cim.size)
#     plt.imshow(cim)
#     plt.show()
    im = image.img_to_array(im)[:,:,0]
    im = cv2.resize(im, (512, 512)) 
    plt.imsave('./kay_data/stimuli_test_resized/stimuli_test_resized_{0}.png'.format(i), im, cmap='gray')
#     cim.save('./kay_data/stimuli/stimuli_{0}.png'.format(i))

### image colorization

In [ ]:
def deepai_colorization(img_path):
    r = requests.post(
        "https://api.deepai.org/api/colorizer",
        files={
            'image': open(img_path, 'rb'),
        },
        headers={'api-key': ''}
    )
    return r

In [ ]:
for i, im in tqdm_notebook(enumerate(glob.glob('kay_data/stimuli_test/*.png'))):
    r =  deepai_colorization(im)
    response = requests.get(r.json()['output_url'])
    img = Image.open(BytesIO(response.content))
    img.save('./kay_data/stimuli_test_colorized/stimuli_test_{0}_color.png'.format(i))

### super resolution images

In [16]:
def deepai_superes(img_path):
    r = requests.post(
        "https://api.deepai.org/api/torch-srgan",
        files={
            'image': open(img_path, 'rb'),
        },
        headers={'api-key': ''}
    )
    return r

In [17]:
for i, im in tqdm_notebook(enumerate(glob.glob('kay_data/stimuli_colorized/*.png'))):
    r = deepai_superes(im)
    response = requests.get(r.json()['output_url'])
    img = Image.open(BytesIO(response.content))
    img.save('./kay_data/stimuli_colorized_superes/stimuli_{0}_color_superes.png'.format(i))

C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


### png to jpeg

In [18]:
for i, im in tqdm_notebook(enumerate(glob.glob('kay_data/stimuli_colorized_superes/*.png'))):
    im = Image.open(im)
    rgb_im = im.convert('RGB')
    rgb_im.save('./kay_data/stimuli_colorized_superes_jpeg/stimuli_{0}_color_superes.jpg'.format(i))

C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


### extract stimuli features

In [ ]:
# https://tech.zegami.com/comparing-pre-trained-deep-learning-models-for-feature-extraction-c617da54641

In [19]:
stimuli_features = dict()

In [3]:
# model = ResNet50(weights='imagenet')
# model = VGG16(weights='imagenet') #, include_top=False)
model = NASNetLarge(weights='imagenet') #, include_top=False)
# model = NASNetMobile(weights='imagenet') #, include_top=False)
# model = MobileNetV2(weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
model_ex = NASNetLarge(weights='imagenet', include_top=False, pooling='avg')

In [5]:
def crop_center(img,cropx,cropy):
    y,x, _ = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [20]:
# img classification

for i, img_path in tqdm_notebook(enumerate(glob.glob('kay_data/stimuli_colorized_superes_jpeg/*.jpg'))):
    try:
#         if i > 20: break
        img_name = img_path.split(sep=os.sep)[1].split(sep='.')[0]
        stimuli_features[img_name] = list()
        img = image.load_img(img_path, target_size=(512, 512))
        x = image.img_to_array(img)
        x = np.array(x)
        x = crop_center(x, 331, 331)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
#         x = x[:,:,:,:]
        preds = model.predict(x)
    #     print('Predicted:', decode_predictions(preds, top=3)[0])
        p = decode_predictions(preds, top=1)[0]
    #     if p[2] >= 0.8:
#         print(p)
        features = model_ex.predict(x)[0]
        stimuli_features[img_name] += [p[0], features]
#         plt.imshow(img)
#         plt.show()
    except:
        print('bad input: ', img_path)

C:\Users\zkalyuzhner\.conda\envs\neurot\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
with open('kay_data/stimuli_features.pickle', 'wb') as fp:
    pickle.dump(stimuli_features, fp, protocol=pickle.HIGHEST_PROTOCOL)